This is the notebook to build the data processing pipeline to feed into the GRU network. It starts with a slimmed and skimmed csv file with p2_calib being the value to predict, and int_deliv_inv_ub and calib_time being the best values to use. It's important to note that at this point, some of the lumi measurements are taken a bit far from the calibration times. this can be seen by the difference between the lumi-section time and the calib_time values. The best data probably just consistents of the points where these are close.

All data is in its original units.

Also, I'm running this in the Python 3.7.1 Kernel

In [1]:
#import the stuff
import pandas as pd #dataframes etc
import matplotlib.pyplot as plt #plotting
import numpy as np
from sklearn import preprocessing
from common.utils import TimeSeriesTensor, create_evaluation_df, mape, scale_shrinker

In [2]:
#load the data
df = pd.read_csv('../scripts/slimmed_df.csv')
#there are many times when the xtal is not fired upon. This means the laser_datetime is NaN, 
#and importantly, the lumi columns: int_deliv_inv_ub and int_record_inv_ub give an erroneous value.
print(df.shape)
df_laser_dropped = df.dropna(subset=['laser_datetime'])
print(df_laser_dropped.shape)

(92170, 22)
(77380, 22)


In [3]:
df_laser_dropped.head()

,Unnamed: 0,Unnamed: 1,iov_idx,seq_datetime,calibration,inst_lumi,laser_datetime,int_inst_lumi,p1,p2,...,#run:fill,ls,beamstatus,E(GeV),delivered(/ub),recorded(/ub),avgpu,source,int_deliv_inv_ub,int_record_inv_ub
0,54000,0,0,2018-02-07 16:14:44,0.937300,NaN,2018-02-07 16:15:53,0.0,0.937300,0.937300,...,314090:6545,67:0,FLAT TOP,6500,0.052346,0.0,0.0,BCM1F,0.052346,0.0
1,54000,1,1,2018-02-07 17:26:08,0.937845,NaN,2018-02-07 17:28:57,0.0,0.937824,0.937845,...,314090:6545,67:0,FLAT TOP,6500,0.052346,0.0,0.0,BCM1F,0.052346,0.0
2,54000,2,2,2018-02-07 17:50:56,0.936282,NaN,2018-02-07 17:53:45,0.0,0.936460,0.936282,...,314090:6545,67:0,FLAT TOP,6500,0.052346,0.0,0.0,BCM1F,0.052346,0.0
3,54000,3,3,2018-02-07 18:15:44,0.936760,NaN,2018-02-07 18:18:33,0.0,0.936706,0.936760,...,314090:6545,67:0,FLAT TOP,6500,0.052346,0.0,0.0,BCM1F,0.052346,0.0
4,54000,4,4,2018-02-07 18:57:17,0.937384,NaN,2018-02-07 19:00:07,0.0,0.937342,0.937384,...,314090:6545,67:0,FLAT TOP,6500,0.052346,0.0,0.0,BCM1F,0.052346,0.0


In [5]:
#the data has multiple possible indices. The most important, is to split the data based on the crystal index which is called Unnamed: 0 at the moment.
df_laser_dropped = df_laser_dropped.rename(columns={'Unnamed: 0':'xtal_idx', 'Unnamed: 1':'dst_idx'})
xtal_groups = df.groupby(df_laser_dropped.xtal_idx)
xtal_list = df_laser_dropped.xtal_idx.unique()


In [7]:
for xtal in xtal_list:
    df = xtal_groups.get_group(xtal)
    print('Extracting crystal: {}'.format(xtal))
    #now we'll select just the columns we want to study
    df_slim = df.loc[:,['laser_datetime','p2','int_deliv_inv_ub']]
    #set the index to the sequence start time
    df_slim['laser_datetime'] = df_slim['laser_datetime'].astype('datetime64')
    df_slim.set_index('laser_datetime', inplace=True)
    #split the data into training, testing and validating
    #select the data range (we'll select the part of the year when the beam was operating)
    data_begin = '2018-05-12 08:00:00'
    data_end = '2018-12-02 16:00:00'

    df_select = df_slim[(df_slim.index > data_begin) & (df_slim.index < data_end)]
    df_select.to_csv('../data/select.csv')
    #select the time ranges for train,valid,test datasets
    df_train_end = '2018-08-31 00:00:00'
    df_valid_end = '2018-10-15 00:00:00'
    df_test_end = data_end

    df_train = df_select[df_select.index < df_train_end]
    df_valid = df_select[(df_select.index < df_valid_end) & (df_select.index > df_train_end)]
    df_test = df_select[(df_select.index < df_test_end) & (df_select.index > df_valid_end)]

    #now that we've looked at the raw data, we have to do a few manipulations.
    #the first is to convert the data into an evenly spaced timeseries. We'll use a rolling average
    import traces as ts

    sampling=3600
    avg_window = sampling*6
    #windows are in seconds
    train_calib_avg = ts.TimeSeries(df_train['p2'])\
                                    .moving_average(sampling, window_size=avg_window, pandas=True)
    train_lumi_avg = ts.TimeSeries(df_train['int_deliv_inv_ub'])\
                                    .moving_average(sampling, window_size=avg_window, pandas=True)

    #now that we've explored the interpolation and made interpolated training data,
    #we'll follow the same steps for the testing and validation data
    valid_calib_avg = ts.TimeSeries(df_valid['p2'])\
                                    .moving_average(sampling, window_size=avg_window, pandas=True)
    valid_lumi_avg = ts.TimeSeries(df_valid['int_deliv_inv_ub'])\
                                    .moving_average(sampling, window_size=avg_window, pandas=True)

    test_calib_avg = ts.TimeSeries(df_test['p2'])\
                                    .moving_average(sampling, window_size=avg_window, pandas=True)
    test_lumi_avg = ts.TimeSeries(df_test['int_deliv_inv_ub'])\
                                    .moving_average(sampling, window_size=avg_window, pandas=True)

    #now we'll put it all back into dataframes
    suffix = '_{}.csv'.format(xtal)
    
    train = pd.DataFrame()
    train['p2'] = train_calib_avg
    train['int_deliv_inv_ub'] = train_lumi_avg
    train.to_csv('../data/train_avg'+suffix)

    valid = pd.DataFrame()
    valid['p2'] = valid_calib_avg
    valid['int_deliv_inv_ub'] = valid_lumi_avg
    valid.to_csv('../data/valid_avg'+suffix)

    test = pd.DataFrame()
    test['p2'] = test_calib_avg
    test['int_deliv_inv_ub'] = test_lumi_avg
    test.to_csv('../data/test_avg'+suffix)

    #now we need to de-trend the data so that it is flat over time
    #for this I'll just apply differencing, since it is pretty simple
    train = train.diff().dropna() #we run dropna, since the first row is NaN in a diff
    valid = valid.diff().dropna()
    test = test.diff().dropna()

    train.to_csv('../data/train_diff'+suffix)
    valid.to_csv('../data/valid_diff'+suffix)
    test.to_csv('../data/test_diff'+suffix)

Extracting crystal: 54000
Extracting crystal: 54001
Extracting crystal: 54002
Extracting crystal: 54003
Extracting crystal: 54004
Extracting crystal: 54005
Extracting crystal: 54006
Extracting crystal: 54007
Extracting crystal: 54008
Extracting crystal: 54009
